In [34]:
import numpy as np

# Урок 6
Взаимосвязь величин. Показатели корреляции. Корреляционный анализ. Проверка на нормальность

## Задача 1

Даны значения величины заработной платы заемщиков банка (salary) и значения их поведенческого кредитного скоринга (scoring):

salary = [35, 45, 190, 200, 40, 70, 54, 150, 120, 110] <br>
scoring = [401, 574, 874, 919, 459, 739, 653, 902, 746, 832]

Используя только встроенные питоновские функции и структуры данных (т.е. без библиотек numpy, pandas и др.) найдите:
1. ковариацию (смещённую и несмещённую) этих двух величин,
2. коэффициент корреляции Пирсона,
3. коэффициент корреляции Кендалла.

Можно затем посчитать те же значения с использованием библиотек, чтобы проверить себя.

In [2]:
salary = [35, 45, 190, 200, 40, 70, 54, 150, 120, 110] 
scoring = [401, 574, 874, 919, 459, 739, 653, 902, 746, 832]

### Найдем среднее значение для двух выборок

Функция расчета среднего

In [6]:
def mean(array: list) -> float:
    sum = 0
    for item in array:
        sum += item
    return sum/len(array)

Находим средние с помощью функции

In [7]:
x_mean = mean(salary)
y_mean = mean(scoring)
x_mean, y_mean

(101.4, 709.9)

### Найдем выборочный коэффициент ковариации

Формула

$\bar{s}_{XY} = cov(X,Y) = \frac{1}{n} \sum^n_{t=1} (X_t - \bar{X})(Y_t - \bar{Y})$

Функция

In [58]:
def s_bar(array_1: list, array_2: list, x_mean: float, y_mean: float, ddof:int ) -> float:
    sum = 0
    if len(array_1) == len(array_2):
        for item_1, item_2 in zip(array_1, array_2):
            sum += (item_1 - x_mean) * (item_2 - y_mean)
        if ddof == 1:
            return sum/(len(array_1) - 1)
        else:
            return sum/len(array_1)
    else:
        print("Массивы должны быть одной длинны")

#### Получаем значение выборочной несмещенной ковариации 

In [59]:
s_bar(salary, scoring, x_mean, y_mean, 1)

10175.377777777778

#### Для смещенной выборочной ковариации

In [39]:
s_bar(salary, scoring, x_mean, y_mean, 0)

9157.84

Проверяем с помощью numpy

In [40]:
np.cov(salary, scoring, ddof = 1), np.cov(salary, scoring, ddof = 0)

(array([[ 3882.93333333, 10175.37777778],
        [10175.37777778, 33854.32222222]]),
 array([[ 3494.64,  9157.84],
        [ 9157.84, 30468.89]]))

### Коэффициент корреляции Пирсона

Формула

$r_{xy} = \frac{\sum{(x_i - \bar{x})*(y_i - \bar{y})}}{\sum(x_i - \bar{x})^2 * \sum(y_i - \bar{y})^2}$

Можно упрастить, так как у нас уже расчитаны показатели ковариации

$r_{xy} = \frac{cov{X,Y}}{\sigma_x\sigma_y}$

где $\sigma_x, \sigma_y$ - стандартное отклонение от X, Y  соответственно

Функция расчета стандартного отклонения

In [78]:
def std(array: list, mean: float) -> float:
    return pow(sum([pow(item - mean,2) for item in array]) / (len(array) - 1), .5)

In [79]:
std(salary, x_mean)

62.31318747531162

Функция

In [82]:
def r(x_array:list, y_array: list, x_mean: float, y_mean:float) -> float:
    cov = s_bar(x_array, y_array, x_mean, y_mean, 1) 
    std_x = std(x_array, x_mean)
    std_y = std(y_array, y_mean)
    return cov / (std_x * std_y)

### Получаем несмещенный коэффицент корреляции Пирсона для 

In [83]:
r(salary, scoring, x_mean, y_mean)

0.8874900920739162

Проверяем на numpy

In [90]:
numpy.corrcoef(salary, scoring)[0, 1]

0.8874900920739162

### Коэффициент корреляции Кендалла.

Формула

$t = \frac{\sum{P} - \sum{Q}}{\sum{P} + \sum{Q}}$

Функция

In [95]:
from itertools import combinations

def is_concordant(i: tuple, j: tuple) -> bool:
    return (i[0] - j[0]) * (i[1] - j[1]) > 0



Получаем значения коэффициента корреляции Кендала

In [96]:
p, q = 0, 0
for i, j in combinations(zip(salary, scoring), r=2):
    if is_concordant(i, j):
        p += 1 
    else:
        q += 1

t = (p - q) / (p + q)
t

0.9111111111111111

## Задача 2

Измерены значения IQ выборки студентов, обучающихся в местных технических вузах:

131, 125, 115, 122, 131, 115, 107, 99, 125, 111

Известно, что в генеральной совокупности IQ распределен нормально. Найдите доверительный интервал для математического ожидания с надежностью 0.95.

In [99]:
x = np.array([131, 125, 115, 122, 131, 115, 107, 99, 125, 111])

Пусть $X \sim N(\mu, \sigma^2)$ 

Тогда в качестве точененой оценки математического ожидалния $m$  используется выборочное среднее $\bar{x}$, а в качестве оценки дисперсии $\sigma^2$ - исправленная выборочная дисперсия

$S^2 = \frac{1}{n-1}\sum\limits_{i=1}^n(x_i - \bar{x})^2$

Из чего следует, что стандартное отклонение будет $S = \sqrt{S^2}$

Считаем

In [118]:
def Ssq(x_array: list) -> float:
    return (1 / (len(x_array) - 1)) * sum([pow(item - x_array.mean(), 2) for item in x_array])

In [119]:
Ssq(x)

111.21111111111111

In [120]:
def Ssqrt(S:float) ->float:
    return pow(S, .5)

Получаем стандартное отклонение

In [124]:
sd =Ssqrt(Ssq(x))
sd

10.54566788359614

In [ ]:
Проверяем на numpy для 

In [122]:
np.std(x ,ddof=1)

10.54566788359614

Формула доверительного интервала для двустороннего t распределения <br>
$P(\overline{x} - t_1\frac{s}{\sqrt{n}} \leq \mu \leq \overline{x} + t_2\frac{s}{\sqrt{n}}) = 0.95$

Считаем квартили

In [127]:
from scipy.stats import t, norm

t1 = t.ppf(0.025, df=(len(x) - 1))
t2 = t.ppf(0.975, df=(len(x) - 1))
t1, t2

(-2.262157162740992, 2.2621571627409915)

Тогда левая граница

In [128]:
x.mean() + t1 * sd / (len(x) ** .5)

110.55608365158724

Правая граница

In [129]:
x.mean() + t2 * sd / (len(x) ** .5)

125.64391634841274

### Ответ
Доверительный интервал с надежностью $0.95$ будет:
$(110.5561, 125.6439)$

# Задача 3

Известно, что рост футболистов в сборной распределён нормально с известной дисперсией 25. На выборке объёма 27 выборочное среднее составило 174.2. Найдите доверительный интервал для математического ожидания с надёжностью 0.95.

Пусть   $X \sim N(\mu, \sigma^2)$ 

$\sigma^2 = 25$,
$n = 27$,
$\bar{x} = 174.2$,
$\alpha = 0.95$

Тогда получаем формулу для доверительного интервала с известной дисперсией <br> $P(\overline{x} - t_1\frac{\sigma}{\sqrt{n}} \leq \mu \leq \overline{x} + t_2\frac{\sigma}{\sqrt{n}}) = 0.95$

Считаем  нужные квантили 

In [136]:
t1 = norm.ppf(0.025)
t2 = norm.ppf(0.975)
t1, t2


(-1.9599639845400545, 1.959963984540054)

### Ответ
Левая граница

In [139]:
174.2 - 1.96 * ((25 / 27) ** .5)

172.31398912064722

Правая граница 

In [141]:
174.2 + 1.96 * ((25 / 27) ** .5)

176.08601087935276